In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
#import pandas_gbp
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import import_ipynb
import covid_bigquery as cb
import plotly.express as px

import plotly.tools as tls
from plotly.offline import iplot
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)

def fetch_exchange_rates():
    # Fetch exchange rate
    df_exchange_rates = pd.read_csv('exchangerates.csv', delimiter = ',')
    df_exchange_rates['date'] = df_exchange_rates['date'].map(lambda x: datetime.strptime(x, "%d/%m/%Y"))
    df_exchange_rates['rate'] = df_exchange_rates['rate'].map(lambda x: float(x))
    return df_exchange_rates
df_exchange_rates = fetch_exchange_rates()

def lineplot_by_country(
        dataframe, 
        y_axis,
        x_axis = "date",
        query = "country in ('US', 'United Kingdom', 'France', 'Italy', 'Egypt', 'India') ", 
        color='country', 
        width=1200):
    fig = px.line(
            dataframe.query(query),
            x=x_axis, y=y_axis, color=color, width=width)
    fig.show()

importing Jupyter notebook from covid_bigquery.ipynb


In [2]:
covid19 = cb.Covid19()

In [3]:
# Plots ratios calculated above, need to change this to pass column names
def covid_plot_ratio(df_covid, df_exchange_rates, country1, country2, type, exch_ratelabel):
    fig, ax = plt.subplots(figsize=(16,4))
    ax.plot(df_covid['date'], df_covid['ratio'], label="{0}/{1} {2}".format(country1, country2, type))
    df_tmp = df_exchange_rates[df_exchange_rates['code'] == exch_ratelabel]
    ax.plot(df_tmp['date'], df_tmp['rate'], label=exch_ratelabel)
    #ax.set_xlim(0, 200) - doesnt work with dates
    #ax.set_ylim(0, 10)
    ax.xaxis.set_major_locator(MultipleLocator(30))
    ax.xaxis.set_minor_locator(AutoMinorLocator(7))
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    ax.grid(which='minor', color='#CCCCCC', linestyle=':')
    ax.legend()
    #plt.show()
    iplot(tls.mpl_to_plotly(fig))

def plot_covid_forex(covid19, df_exchange_rates, exchrate_code, country1, country2):
    df_ratio = covid19.covid_ratio_by_country('confirmed_per1m',   country1, country2)
    covid_plot_ratio(df_ratio, df_exchange_rates, country1, country2, 'confirmed', exchrate_code)

    df_ratio = covid19.covid_ratio_by_country('deaths_per1m',  country1, country2)
    covid_plot_ratio(df_ratio, df_exchange_rates, country1, country2, 'deaths', exchrate_code)

    df_ratio = covid19.covid_ratio_by_country('active_per1m',  country1, country2)
    covid_plot_ratio(df_ratio, df_exchange_rates, country1, country2, 'active', exchrate_code)

#df_exchange_rates['rate'][df_exchange_rates['code'] == exchrate_label] = df_exchange_rates['rate'][df_exchange_rates['code'] == exchrate_label]/35
plot_covid_forex(covid19, df_exchange_rates, 'USDINR', 'US', 'India')
plot_covid_forex(covid19, df_exchange_rates, 'GBPUSD', 'US', 'United Kingdom')


In [4]:
plot_covid_forex(covid19, df_exchange_rates, 'EURUSD', 'Germany', 'US')
plot_covid_forex(covid19, df_exchange_rates, 'EURUSD', 'France', '')

In [5]:
lineplot_by_country(covid19.df_cumulative, "confirmed")
lineplot_by_country(covid19.df_cumulative, "deaths")
lineplot_by_country(covid19.df_cumulative, "confirmed_per1m")
lineplot_by_country(covid19.df_cumulative, "deaths_per1m")
lineplot_by_country(covid19.df_daily, "New cases")
lineplot_by_country(covid19.df_daily, "New deaths")